# 定義函數

In [3]:
CityName1={"danshui":"新北市淡水區","nangang":"台北市南港區","zhongshan_keelung":"基隆市中山區",
          "renai":"基隆市仁愛區","anle":"基隆市安樂區","neihu":"台北市內湖區",
          "xinzhuang":"新北市新莊區","shulin":"新北市樹林區","banqiao":"新北市板橋區",
          "songshan":"台北市松山區","zhonghe":"新北市中和區","nuannuan":"基隆市暖暖區",
          "qidu":"基隆市七堵區","keelung_zhongzheng":"基隆市中正區","keelung_xinyi":"基隆市信義區",
          "xizhi":"新北市汐止區","taipei_zhongshan":"台北市中山區","datong":"台北市大同區","daan":"台北市大安區",
          "taipei_zhongzheng":"台北市中正區","taipei_xinyi":"台北市信義區","wanhua":"台北市萬華區",
          "shilin":"台北市士林區","beitou":"台北市北投區","luzhou":"新北市蘆洲區","sanchong":"新北市三重區",
          "tucheng":"新北市土城區","yonghe":"新北市永和區"}

CityName=("danshui","nangang","zhongshan_keelung","renai","anle","neihu","xinzhuang","shulin","banqiao",
              "songshan","zhonghe","nuannuan","qidu","keelung_zhongzheng","keelung_xinyi","xizhi","taipei_zhongshan",
             "datong","daan","taipei_zhongzheng","taipei_xinyi","wanhua","shilin","beitou","luzhou","sanchong","yonghe","tucheng")
B_type=("NORMAL","BUSINESS","PIPE")

def mkdir(path):
    import os
    
    path=path.strip()
    path=path.rstrip("\\")
    isExists=os.path.exists(path)

    if not isExists:
        
        print (path+' 創建成功')
        os.makedirs(path)
        return True
    else:
        print (path+' 目錄已存在')
        return False

def autocraw(ccc):
    import pandas as pd
    import psycopg2
    import time

    start = time.clock()
    




    conn=psycopg2.connect(database=ccc, user="postgres", password="12345",host='localhost', port="5432")

    with conn:
        for x in CityName:
            for y in B_type:
                cur=conn.cursor()
                cur.execute("select id,postcode,business_type,city_name,address,latitude,longitude from consumer where city_name='{}' and business_type='{}'".format(x,y)) 
                columns1 = ['id','postcode','business_type','city_name','address','latitude','longitude']
                rows = cur.fetchall()
                z=pd.DataFrame(rows,columns = ['id','postcode','business_type','city_name','address','latitude','longitude'] )
               # z.index.name = 'dd'  #原本ˋpandas會自動產生一列index,故先定義index column名為dd,之後再刪除.
                cur.close()

                writer = pd.ExcelWriter('maps\{}\{}_{}.xlsx'.format(CityName1[x],CityName1[x],y), engine='xlsxwriter')
                z.to_excel(writer, 'Sheet1')
                writer.save()  
    elapsed = (time.clock() - start)
    print("Time used:",elapsed)
    

    
def data_clean(): 
    import time
    import pandas as pd
    start = time.clock()

    
    for x in CityName:
        for y in B_type:
            df = pd.read_excel('./maps\{}\{}_{}.xlsx'.format(CityName1[x],CityName1[x],y)) #讀檔
            df.loc[df['city_name']=='{}'.format(x), 'city_name'] = CityName1["{}".format(x)] #將英文區域改成中文
            df["full_address"] = df["city_name"]+df["address"] #合併2欄
            df = df.drop(['address'],axis=1).drop(['city_name'],axis=1) #將city_name跟address2欄刪除

            #將column(full_address)從欄5更改到欄2
            full_address = df['full_address']
            df.drop(labels=['full_address'], axis=1,inplace = True)
            df.insert(2, 'full_address', full_address)
            writer = pd.ExcelWriter('maps\{}\{}_{}.xlsx'.format(CityName1[x],CityName1[x],y), engine='xlsxwriter')
            df.to_excel(writer, 'Sheet1')
            writer.save()          

        elapsed = (time.clock() - start)
    print("Time used:",elapsed)
  


# 執行

In [4]:
for y in CityName1.values():
    mkdir("maps\{}".format(y)) 

autocraw("prod_20180503")
data_clean()

maps\新北市淡水區 創建成功
maps\台北市南港區 創建成功
maps\基隆市中山區 創建成功
maps\基隆市仁愛區 創建成功
maps\基隆市安樂區 創建成功
maps\台北市內湖區 創建成功
maps\新北市新莊區 創建成功
maps\新北市樹林區 創建成功
maps\新北市板橋區 創建成功
maps\台北市松山區 創建成功
maps\新北市中和區 創建成功
maps\基隆市暖暖區 創建成功
maps\基隆市七堵區 創建成功
maps\基隆市中正區 創建成功
maps\基隆市信義區 創建成功
maps\新北市汐止區 創建成功
maps\台北市中山區 創建成功
maps\台北市大同區 創建成功
maps\台北市大安區 創建成功
maps\台北市中正區 創建成功
maps\台北市信義區 創建成功
maps\台北市萬華區 創建成功
maps\台北市士林區 創建成功
maps\台北市北投區 創建成功
maps\新北市蘆洲區 創建成功
maps\新北市三重區 創建成功
maps\新北市土城區 創建成功
maps\新北市永和區 創建成功
Time used: 3.371921690061612
Time used: 3.9420256463819427
